In [1]:
!pip install kaggle



In [2]:
from pathlib import Path


In [3]:
from pathlib import Path

path = Path('us-patent-phrase-to-phrase-matching')

if  not path.exists():
    import zipfile,kaggle
    kaggle.api.competition_download_cli(str(path))
    zipfile.ZipFile(f'{path}.zip').extractall(path)


In [4]:


!ls {path}




sample_submission.csv  test.csv  train.csv


In [5]:
import pandas as pd

In [6]:
df = pd.read_csv(path/'train.csv')


In [7]:
df

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [8]:
df.describe(include='object')


,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,8d135da0b55b8c88,component composite coating,composition,H01
freq,1,152,24,2186


In [9]:
df['input'] = 'TEXT1: ' + df.context + '; TEXT2: ' + df.target + '; ANC1: ' + df.anchor

In [10]:
from datasets import Dataset,DatasetDict
ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

In [11]:
model_nm = 'microsoft/deberta-v3-small'

In [12]:


from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm)



/home/prinzz/main/my-projects/fastai/virtual-env/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [13]:
def tok_func(x): return tokz(x["input"])


tok_ds = ds.map(tok_func, batched=True)



Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

In [14]:
row = tok_ds[0]
row['input'], row['input_ids']


('TEXT1: A47; TEXT2: abatement of pollution; ANC1: abatement',
 [1,
  54453,
  435,
  294,
  336,
  5753,
  346,
  54453,
  445,
  294,
  47284,
  265,
  6435,
  346,
  23702,
  435,
  294,
  47284,
  2])

In [15]:


tok_ds = tok_ds.rename_columns({'score':'labels'})



In [16]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

In [17]:
eval_df = pd.read_csv(path/'test.csv')
eval_df['input'] = 'TEXT1: ' + eval_df.context + '; TEXT2: ' + eval_df.target + '; ANC1: ' + eval_df.anchor
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

In [18]:
import numpy as np


def corr(x,y): return np.corrcoef(x,y)[0][1]
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

In [19]:
from transformers import TrainingArguments,Trainer

In [20]:
bs = 128
epochs = 4
lr = 8e-5

In [21]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    eval_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

In [22]:


model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=corr_d)



Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_883/2945279256.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
/home/prinzz/main/my-projects/fastai/virtual-env/lib/python3.12/site-packages/accelerate/accelerator.py:449: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [23]:
trainer.train();

Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.034294,0.797424
2,No log,0.023484,0.822537
3,0.050900,0.024343,0.832712
4,0.050900,0.023224,0.832649


In [24]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds

array([[ 0.57568359],
       [ 0.69287109],
       [ 0.59667969],
       [ 0.32714844],
       [-0.00822449],
       [ 0.55859375],
       [ 0.53271484],
       [ 0.00553513],
       [ 0.29150391],
       [ 1.06640625],
       [ 0.24609375],
       [ 0.25024414],
       [ 0.82861328],
       [ 0.80273438],
       [ 0.77636719],
       [ 0.46606445],
       [ 0.34301758],
       [-0.03634644],
       [ 0.59863281],
       [ 0.35742188],
       [ 0.46044922],
       [ 0.26635742],
       [ 0.1015625 ],
       [ 0.24560547],
       [ 0.59423828],
       [-0.03427124],
       [-0.00583649],
       [-0.03939819],
       [-0.03683472],
       [ 0.61816406],
       [ 0.31005859],
       [ 0.0300293 ],
       [ 0.72558594],
       [ 0.50878906],
       [ 0.48168945],
       [ 0.27246094]])

In [25]:
preds = np.clip(preds, 0, 1)

In [26]:
import datasets

submission = datasets.Dataset.from_dict({
    'id': eval_ds['id'],
    'score': preds
})

submission.to_csv('submission.csv', index=False)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

1031